# UpSet.js Jupyter Widget

[UpSet.js](https://upset.js.org) is a JavaScript re-implementation of [UpSetR](https://www.rdocumentation.org/packages/UpSetR/) which itself is based on [UpSet](http://vcg.github.io/upset/about/). 
The core library is written in React but provides also bundle editions for plain JavaScript use and this Jupyter wrapper. 

In this tutorial the basic widget functionality is explained.

Let's begin with importing the widget and some utilities

In [18]:
from ipywidgets import interact
from upsetjs_jupyter_widget import UpSetWidget
import pandas as pd

This wrapper is implemented in Python 3 with mypy typings and generics. The generic type `T` of the `UpSetWidget` is type of element that we wanna handle. In the following example we handle `str` elements.

In [19]:
w = UpSetWidget[str]()

## Basic User Interface

**Note**: The input data will be described in more detail in the next section

In [20]:
dict_input = dict(one = ['a', 'b', 'c', 'e', 'g', 'h', 'k', 'l', 'm'], two = ['a', 'b', 'd', 'e', 'j'], three = ['a', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm'])
w.from_dict(dict_input)

UpSetWidget(value=None, elems=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm'], height_ratios…

UpSet consists of three areas:

- The bottom left area shows the list of sets as a vertical bar chart. The length of the bar corresponds to the cardinality of the set, i.e., the number of elements in this set. 
- The top right area shows the list of set intersections as a horiztonal bar chart. Again the length corresponds to the cardinality of the set
- The bottom right area shows which intersection consists of which sets. A dark dot indicates that the set is part of this set intersection. The line connecting the dots is just to visually group the dots.

Moving the mouse over a bar or a dot will automatically highlight the corresponding set or set intersection in orange. In addition, the number elements which are shared with the highlighted sets are also highlighted. This gives a quick overview how sets and set intersections are related to each other. More details, in the [Interaction](#intersection) section.

In the bottom right corner there are two buttons for exporting the chart in either PNG or SVG image format.

## Input Formats

In the current version the UpSet wrapper supports two input data formats: `dictionary` and through a Pandas `dataframe`.

### Dictionary Input

The first format is a dictionary of type `Dict[str, List[T]]`, `T` refers again to the elements type, in this case it is a list of `str`. The key of the dictionary entry is the set name while the value is the list of elements this set has.

In [21]:
w.from_dict(dict(one = ['a', 'b', 'c', 'e', 'g', 'h', 'k', 'l', 'm'], two = ['a', 'b', 'd', 'e', 'j'], three = ['a', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm']))

UpSetWidget(value=None, elems=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm'], height_ratios…

### Data Frame Input

The second format is a a binary/boolean data frame. The index column contains the list of elements. Each regular color represents a sets with boolean values (e.g., 0 and 1) whether the row represented by the index value is part of the set or not.

The following data frame defines the same set structure as the dictionary format before.

In [22]:
df = pd.DataFrame(dict(
    one=[1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1], 
    two=[1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0], 
    three=[1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1]
), index=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm'])
w.from_dataframe(df)

UpSetWidget(value=None, elems=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm'], height_ratios…

## Basic Attrributes

`.elems` returns the list of extracted elements

In [23]:
w.elems

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm']

`.sets` returns the list of extracted sets as `UpSetSet` objects

In [24]:
w.sets

[UpSetSet(name=one, elems={'e', 'g', 'c', 'a', 'h', 'm', 'b', 'k', 'l'}),
 UpSetSet(name=three, elems={'i', 'e', 'h', 'g', 'a', 'f', 'j', 'm', 'l'}),
 UpSetSet(name=two, elems={'j', 'b', 'e', 'a', 'd'})]

Similariy, `.combinations` returns the list of set intersections that are visualized as `UpSetIntersection` objects. 

**Note**: the attribute is called `.combinations` instead of `.intersections` since one customized the generation of the set combinations that are visualized. E.g., one can also generate set unions.

In [25]:
w.combinations

[UpSetSetIntersection(name=one, sets={'one'}, elems={'e', 'g', 'c', 'a', 'h', 'm', 'b', 'k', 'l'}),
 UpSetSetIntersection(name=three, sets={'three'}, elems={'i', 'e', 'h', 'g', 'a', 'f', 'j', 'm', 'l'}),
 UpSetSetIntersection(name=(one ∩ three), sets={'three', 'one'}, elems={'m', 'e', 'g', 'h', 'a', 'l'}),
 UpSetSetIntersection(name=two, sets={'two'}, elems={'j', 'b', 'e', 'a', 'd'}),
 UpSetSetIntersection(name=(one ∩ two), sets={'two', 'one'}, elems={'a', 'b', 'e'}),
 UpSetSetIntersection(name=(three ∩ two), sets={'three', 'two'}, elems={'a', 'j', 'e'})]

`.generate_intersections` and `.generate_unions` let you customize the generation of the set combinations

 - `min_degree` ... minimum number of sets in a set combination
 - `max_degree` ... maximum number of sets in a set combination, `None` means no limit
 - `empty` ... include empty set combinations with no elements. By default they are not included
 - `order_by` ... sort set combinations either by `cardinality` (number of elements) or by `degree` (number of sets
 - `limit` ... show only the first `limit` set combinations

In [26]:
w.copy().generate_intersections(min_degree=2, max_degree=None, empty=True, order_by="cardinality", limit=None)

UpSetWidget(value=None, elems=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm'], height_ratios…

In [27]:
w.copy().generate_unions(min_degree=0, max_degree=2, empty=True, order_by="degree", limit=None)

UpSetWidget(value=None, elems=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm'], height_ratios…

## Interaction

UpSet allows three intersection modes settable via `.mode`

- `'hover'` (default) when the user hovers over a set or set intersection it will be highlighted. This is the default mode
- `'click'` when the user clicks on a set or a set intersection, the selection will be updated
- `'static'` disables interactivity

In [28]:
w.mode = 'click'
w

UpSetWidget(value=None, elems=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm'], height_ratios…

with `.selection` one manually sets the selection that is currently highlighted. Manually setting the selection is only useful in `click` and `static` modes.

In [29]:
w.selection = w.sets[0]
w

UpSetWidget(value={'type': 'set', 'name': 'one'}, elems=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k'…

The current selection is synced with the server. It is designed to work with the `interact` of the `ipywidgets` package. In the following example the current selected set will be automatically written below the chart and updated interactivly.

In [30]:
w.mode = 'hover'
def selection_changed(s):
    return s # s["name"] if s else None
interact(selection_changed, s=w)

interactive(children=(UpSetWidget(value={'type': 'set', 'name': 'one'}, elems=['a', 'b', 'c', 'd', 'e', 'f', '…

<function __main__.selection_changed(s)>

## Queries

besides the selection UpSet supports defining queries. A query can be a list of elements or a set that should be highlighted. A query consists of a name, a color, and either the list of elements or the upset set (combination) to highlight.

In [31]:
wq = w.copy()
wq.mode = 'static'
wq.selection = None
wq.append_query('Q1', color='red', elems=['a', 'b', 'c'])
wq.append_query('Q1', color='blue', upset=wq.sets[1])
wq

UpSetWidget(value=None, elems=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm'], height_ratios…

## Attributes

UpSet.js supports rendering boxplots as aggregations for numerical attributes of elements. The are given as part of the data frame. The attributes element can either have a list of column names or a data frame with the same index

In [32]:
df_a = pd.DataFrame(dict(
    one=[1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1], 
    two=[1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0], 
    three=[1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1],
    attr=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
), index=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm'])
wa = w.copy()
wa.from_dataframe(df_a, attributes=['attr'])
wa

UpSetWidget(value=None, attrs=OrderedDict([('attr', [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])]), elems=['a'…

## Styling

### Theme

UpSet.js supports two themes: `light` and `dark`. The theme can be set by the `.theme` property

In [ ]:
w_dark = w.copy()
w_dark.theme = 'dark'
w_dark

### Labels

In [ ]:
w_label = w.copy()
w_label.set_name = 'Set Label'
w_label.combination_name = 'Combination Label'
w_label

### Log Scale

setting `.numerical_scale = 'log'` switches to a log scale, similarly `'linear'` goes back to a linear scale

In [ ]:
w_log = w.copy()
w_log.numeric_scale = 'log'
w_log

### Size

the `.width` and `.height` properties can be used to specify the width and height of the chart respectively. In general, the `.layout` of the Jupyter Widgets can be used to customize it.

In [ ]:
w.height = 600
w